# Real-time Bus Tracking Visualization 

In [38]:
# !pip install --upgrade gtfs-realtime-bindings
# !pip install json
# !pip install folium
# !pip install geopy
# !pip install selenium
# !pip install html2image

In [1]:
from google.transit import gtfs_realtime_pb2
import requests
import folium
from geopy.geocoders import Nominatim
import json
import time
import datetime
import os
from html2image import Html2Image
import cv2

The provided code is a Python function `get_map(time)` that retrieves real-time bus data using the Open Transit Data API and visualizes it on a map. Here's a summary:

1. The code fetches the real-time bus data from the API and converts it into a JSON string.
2. Latitude and longitude coordinates of the buses are extracted from the JSON data and stored in a list.
3. Using the Folium library, a map centered around New Delhi is created, and small red circle markers are added to represent the bus locations.
4. The resulting map is saved as an HTML file, with the filename based on the provided `time` parameter (this is only for making the filename unique and also storing the time for future references).

This code provides a basic framework for visualizing real-time bus tracking and can be further customized and expanded based on your specific needs and preferences.

In [3]:
def get_map(time):
  feed = gtfs_realtime_pb2.FeedMessage()
  k = "https://opendata.iiitd.edu.in//api//realtime//VehiclePositions.pb?key={your key here}"
  response = requests.get(k)
  feed.ParseFromString(response.content)
  from google.protobuf.json_format import MessageToJson
  json_string = MessageToJson(feed)
  d = eval(json_string)
  l = []
  for i in d["entity"]:
    l.append([i["vehicle"]["position"]["latitude"],i["vehicle"]["position"]["longitude"]])
  geolocator = Nominatim(user_agent="location_details")
  location = geolocator.geocode("New Delhi")
  latitude = location.latitude
  longitude = location.longitude

  # Create a map centered around the city
  m = folium.Map(location=[latitude, longitude], zoom_start=10)


  map_new = folium.Map(location=[location.latitude, location.longitude], zoom_start=10.5, width=1920, height=1080)

  for i, j in l:
      folium.CircleMarker(location=[i, j],
                          radius=0.01,
                          color='red',
                          fill=True,
                          fill_color='red',
                          fill_opacity=1).add_to(map_new)

  map_new.save('htmlfiles\\'+'map'+str(time)+'.html')


In [5]:
while True:
    try:
        get_map(time.time())
    except:
        pass
    time.sleep(30)

### This code is to convert the html file to a png file, it basically takes a screenshot of that html page, I was not able to find any better solution for this.

In [14]:
hti = Html2Image()
path = "html files path"

cnt = 1
for i in os.listdir(path):
    new = os.path.join(path, i)
    f = open(new, "r")
    html_str = f.read()
    hti.screenshot(html_str=html_str, save_as="time_"+str(cnt)+".png")
    cnt += 1
    f.close()


### This code is used to make the video using the images where each frame represents a image

In [ ]:
image_directory = 'images directory path'
image_files = [f for f in os.listdir(image_directory) if os.path.isfile(os.path.join(image_directory, f))]

image_files.sort()
width, height = 1920, 1080
fps = 28
video_writer = cv2.VideoWriter('output_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
for image_file in image_files:
    image_path = os.path.join(image_directory, image_file)
    image = cv2.imread(image_path)
    image = cv2.resize(image, (width, height))
    video_writer.write(image)
video_writer.release()